In [ ]:
# Este codigo lee modelos metabolicos SBML y extrae todas las reacciones y sus genes asociados,
# para analizar las reglas logicas GPR y calcular el numero minimo de deleciones geneticas para perder la reaccion
# Genera un CSV con toda la informacion (cada fila es una reaccion de cada modelo)


# CODIGO PARA GENERAR TABLA MAESTRA
# SETUP: instalar cobrapy y programas necesarios:
print("Instalando...")
!pip install -q "cobra==0.29.0" "optlang==1.8.3" "python-libsbml==5.20.2" "swiglpk>=5.0.5"

# -q = quiet, mutea msjes de pip
# cobra =  modelado metabolico escala genomica (cargar mdoelos, hacer FBA, etc)
# optlang = plantea y resuelve problema optimizacion lineal
# python-libsbml = parsear archivos sbml (leerlos)
# swiglpk = glpk solver que resuelve LP por defecto para optimizar FBA

# todo este pipeline (sbmls > libsbml > cobra > optlang > solver > resultados) permite VINCULAR GENES CON REACCIONES en cada modelo
# asi se mapea q genes estan presente en cada modelo, cuantos genes controlan rxns, etc

# montar Google Drive para acceder a archivos
print("Montando Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# verificar instalaciones
print("Setup completado")

Instalando...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 40.9 MB/s eta 0:00:00
Montando Google Drive...
Mounted at /content/drive
Setup completado


In [ ]:
# CONFIGURACION Y FUNCIONES UTILES -------------------------

# importar librerias:
from cobra import io
# libreria para leer modelos metabolicos
import os, glob, pandas as pd, logging, re
from tqdm.auto import tqdm
from itertools import combinations
import warnings
from datetime import datetime # guardar carpetas separadas

# glob: buscar archivos cn caracteres dados, patrones
# pandas: manejo tablas/datos
# logging: mensajes de info/errores (logs)
# tqdm: barra de progreso
# combinations: probar combinaciones de genes eliminados

# silenciar logs y warnings (no obligatorio pero RECOMENDADO), ahorra tiempo
warnings.filterwarnings('ignore')
logging.getLogger('cobra').setLevel(logging.CRITICAL)
logging.getLogger('cobra.io').setLevel(logging.CRITICAL)

# configurar rutas
CARPETA_MODELOS = "/content/drive/MyDrive/MASH_primavera_2025/modelos_sbml" # ubicar los 211 modelos
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") # distinguir carpeta salida x fecha y hora
CARPETA_SALIDA = f"/content/drive/MyDrive/MASH_primavera_2025/out/run_{timestamp}"
os.makedirs(CARPETA_SALIDA, exist_ok=True) # carpeta salida sin sobreescribir

SALIDA_CSV = os.path.join(CARPETA_SALIDA, "reacciones_completo.csv")

# buscar archivos SBML, orden alfabetico
# glob.glob busca archivos q coinciden con patron dado (al10.sbml, al22.sbml, etc):
paths = sorted(glob.glob(os.path.join(CARPETA_MODELOS, "*.sbml")))

# mostrar lista de archivos encontrados:
print(f"SBML encontrados: {len(paths)}") # cuantos sbml encontro
print(f" Salida: {CARPETA_SALIDA}")

# FUNCIONES HELPER -------------------------

### normalizar GPR (Gene Protein Reaction): evita problemas de parseo (espacios entre genes normalizados)
# elimina espacios extra, tabs, etc
def normaliza_gpr(gpr: str) -> str:
    return re.sub(r"\s+", " ", (gpr or "").strip())

### convertir reaccion a string (Ej: gluocsa:-1, piruvato: 2, ATP: -1 => 1 glucosa + 1 ATP -> 2 piruvato):
# sirve para legibilidad
def to_esteq_str(rxn) -> str:
    try:
        return rxn.reaction # obtener string legible
    except: # si falla, construye string manualmente
        lhs = " + ".join(f"{-c} {m.id}" for m,c in rxn.metabolites.items() if c < 0)
        rhs = " + ".join(f"{c} {m.id}" for m,c in rxn.metabolites.items() if c > 0)
        arrow = "<->" if rxn.reversibility else "->"
        return f"{lhs} {arrow} {rhs}"
        # lhs: reactivos (left, coef negativo)
        # rhs: productos (right, coef positivo)

### funcion para convertir una regla GPR (combinaciones AND, OR) a una lista de conjunto de genes
def dnf_clauses(gpr: str): # dnf: expresion logica
    if not gpr:
        return [] # si regla GPR esta vacia, retornar lista vacia

    # necesario para leer bien los parentesis:
    expr = gpr.lower() # pasar AND, OR a minusculas
    partes, nivel, buf, i = [], 0, [], 0
    # partes: lista que guarda pedazos finales seoarados x OR
    # nivel: numero que cuenta cuantos parentesis abiertos en momento actual, sube y baja
    # buf: lista q acumula temporalmente carcateres q pertenecen a lo q leo (antes del prox OR)
    # i es numero q representa indice/posicion actual dentro del string

    # Separar por OR respetando paréntesis
    while i < len(expr): # recorrer cadena
        ch = expr[i] # caracter actual
        if ch == "(": # si carcater i es parentesis (, sube numero nivel
            nivel += 1 # nivel ve cuantos parentesis abiertos hay
            buf.append(ch)
        elif ch == ")": # si es ), cierro ese trozo, salgo un nivel
            nivel = max(0, nivel-1) # evita quedar negativo
            buf.append(ch)
        elif nivel == 0 and expr[i:i+4] == " or ": # REVISAR ESTO !!!!!!!
            partes.append("".join(buf).strip())
            buf = []
            i += 3
        else:
            buf.append(ch) # si no pasa lo anterior, acumulo carcater en buf
        i += 1 # avanzar cadena

    if buf:
        partes.append("".join(buf).strip())

    # convertir cada parte en conjunto de genes/clausula
    clausulas = []
    for p in partes:
        p = p.strip()
        if p.startswith("(") and p.endswith(")"):
            p = p[1:-1].strip()
        genes = [g.strip("() ") for g in p.split(" and ") if g.strip()]
        clausulas.append(set(genes))

    return clausulas # devuelve lista de clausulas

# Ejemplos de lo anterior:
# input:  "(A and B) or C" -> salida: [{"a","b"}, {"c"}]
# input:  "A or B or C" -> salida: [{"a"}, {"b"}, {"c"}]
# input:  "A and B and C" -> salida: [{"a","b","c"}]

# OBS:
# Si la GPR viene como "A or(B and C)", "A OR B", "A Or B", "A||B"... no la va a partir bien.
# caso critico: (A and (B or C)) -> deberia reescribir (A and B) or (A and C)  →  [{"a","b"}, {"a","c"}]
# normalizar con regex (?) para poner espacios alrededor de operadores

## FUNCION para ver cuantos genes hay q eliminar para perder una reaccion (importante)
def min_deleciones_para_inactivar(gpr: str):
    if not gpr:
        return None

    clauses = dnf_clauses(gpr) # obtiene las clausulas
    if not clauses:
        return None

    # Cota inferior: tamaño de la cláusula más pequeña
    lb = min(len(c) for c in clauses if c)

    # Genes totales
    universo = sorted(set().union(*clauses))
    LIM = min(6, len(universo))  # Límite computacional  REVISAR ESTO !!

    # Buscar combinación mínima que inactiva todas las cláusulas
    for k in range(lb, LIM+1):
        for combo in combinations(universo, k):
            S = set(combo)
            # Si todas las cláusulas pierden al menos 1 gen → reacción inactiva
            if all(len(c & S) >= 1 for c in clauses):
                return k

    return lb  # Retorna cota inferior si no encuentra solución exacta

# Ejemplos de lo anterior:
# GPR: "gene1 or (gene2 and gene3)"
# Cláusulas: [{gene1}, {gene2, gene3}]
# Resultado: 1 (eliminando gene1 se inactiva todo)

print("Funciones helper cargadas")

SBML encontrados: 211
 Salida: /content/drive/MyDrive/MASH_primavera_2025/out/run_20251225_171231
Funciones helper cargadas


In [ ]:
# procesamiento de 211 modelos sbml y generacion CVS con info rxnes y redundancia
# procesar todos los 211 modelos sbml, extrae reacciones y sus genes
# calcula redundancia y crea dataframe: CSV con info genes y redundancia

# crear carpeta de salida si no existe
os.makedirs(os.path.dirname(SALIDA_CSV), exist_ok=True)

# definir columnas del CSV
cols = [
    "modelo_id",           # Identificador del modelo (ej: al10, top4)
    "sitio",
    "rxn_id",              # ID de la reacción
    "rxn_nombre",          # Nombre descriptivo
    "estequiometria",      # String legible de la reacción
    "gpr",                 # Regla lógica Gene-Protein-Reaction
    "genes",               # Lista de genes (separados por coma)
    "subsystem",           # Vía metabólica (ej: Glycolysis)
    "ec",                  # Código EC de la enzima
    "min_deleciones_gpr"   # Métrica de redundancia
]

# crear archivo CSV con encabezado
with open(SALIDA_CSV, "w") as f:
    f.write(",".join(cols) + "\n")

# procesar cada modelo
print(f"Procesando {len(paths)} modelos...\n")

for ruta in tqdm(paths, desc="Modelos", unit="modelo"): # itera x modelo
    try:
        # extraer ID del modelo
        modelo_id = os.path.splitext(os.path.basename(ruta))[0]

        # extraer el sitio/localidad: al, chon, etc
        sitio = re.match(r'^([a-z]+)', modelo_id).group(1) if re.match(r'^([a-z]+)', modelo_id) else "desconocido"

        # cargar modelo SBML
        m = io.read_sbml_model(ruta)
        filas = []

        # iterar sobre cada reacción
        for rxn in m.reactions:
            gpr = normaliza_gpr(rxn.gene_reaction_rule)
            genes = ",".join(sorted(g.id for g in rxn.genes))
            subsystem = getattr(rxn, "subsystem", "")

            # extraer código EC (primero desde la anotación)
            ec = None
            if isinstance(rxn.annotation, dict):
                ec = rxn.annotation.get("ec-code") or rxn.annotation.get("ec_number")

            # si la anotación no tiene EC, lo intentamos extraer desde el rxn_id
            # ejemplo: 1.1.1.1-RXN → EC = 1.1.1.1
            if not ec:
                match = re.search(r'\d+\.\d+\.\d+\.\d+', rxn.id)
                if match:
                    ec = match.group(0)


            # construir fila con info
            filas.append({
                "modelo_id": modelo_id,
                "sitio": sitio,
                "rxn_id": rxn.id,
                "rxn_nombre": rxn.name,
                "estequiometria": to_esteq_str(rxn),
                "gpr": gpr,
                "genes": genes,
                "subsystem": subsystem if subsystem else "",
                "ec": ec if ec else "",
                "min_deleciones_gpr": min_deleciones_para_inactivar(gpr)  # CLAVE
            })

        # guardar fila de este modelo al CSV (modo append, sin borrar contenido q ya tiene)
        pd.DataFrame(filas).to_csv(SALIDA_CSV, mode="a", header=False, index=False)

    except Exception as e:
        print(f"  [WARN] {modelo_id} falló: {e}")

print(f"Archivo guardado en: {SALIDA_CSV}")

# Verificacion resultados:
df = pd.read_csv(SALIDA_CSV) # cargar CSV completo para verificar
print(f"RESUMEN:")
print(f"  • Total de filas: {len(df):,}")
print(f"  • Modelos únicos: {df['modelo_id'].nunique()}") # deberia ser 211
print(f"  • Reacciones únicas: {df['rxn_id'].nunique()}")
print(f"  • Genes totales: {df['genes'].str.split(',').explode().nunique()}")

Procesando 211 modelos...



Modelos:   0%|          | 0/211 [00:00<?, ?modelo/s]

Archivo guardado en: /content/drive/MyDrive/MASH_primavera_2025/out/run_20251225_171231/reacciones_completo.csv
RESUMEN:
  • Total de filas: 319,160
  • Modelos únicos: 211
  • Reacciones únicas: 5871
  • Genes totales: 165071
